This notebooks has a dual purpose:

1. it is is execute in the `germany-hospitalisierungsrate.ipynb` notebook via the `%run NOTEBOOK` magic. So we need to fetch the data here, and make the `plot_hi` function available, as the namespace of the two notebooks is shared.

2. it is also used as a development pad for new features. The commented out commands below record some of the development work, and help to re-start that activity.
 
If we create figures in this notebook, they will be displayed in the `germany-hospitalisierungsrate.ipynb`. Display of tables seems to be suppressed. Output to stdout semes to be  displayed.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime

%config InlineBackend.figure_format = 'svg'

data_url = "https://github.com/robert-koch-institut/COVID-19-Hospitalisierungen_in_Deutschland/blob/master/Aktuell_Deutschland_COVID-19-Hospitalisierungen.csv?raw=true"


In [ ]:
def germany_fetch_hospitalisierungs_inzidenz():

    data = pd.read_csv(data_url)
    data = data.set_index(pd.to_datetime(data["Datum"]))
    # Select a sum over all age groups
    data = data[data["Altersgruppe"] == "00+"]
    return data


In [ ]:
# local copy to work offline
# !wget "https://github.com/robert-koch-institut/COVID-19-Hospitalisierungen_in_Deutschland/blob/master/Aktuell_Deutschland_COVID-19-Hospitalisierungen.csv?raw=true"


In [ ]:
# filter for the data we care about
data = germany_fetch_hospitalisierungs_inzidenz()
# select subset of data we care about
data = data[["Bundesland", "7T_Hospitalisierung_Inzidenz"]]

In [ ]:
# HospitalisierungsInzidenz
hi = data[["Bundesland", "7T_Hospitalisierung_Inzidenz"]]

In [ ]:
hi_bund = hi[hi["Bundesland"] == "Bundesgebiet"]

In [ ]:
def fine_tune_plot(ax):
    fig = ax.get_figure()
    fig.autofmt_xdate()

In [ ]:
## Example:
# fig, ax = plt.subplots()
# ax.plot(hi_bund.index, hi_bund["7T_Hospitalisierung_Inzidenz"].values)
# fine_tune_plot(ax)

In [ ]:
# Overview of current numbers
today = hi.loc[hi.index[0]]


In [ ]:
today.sort_values(by='7T_Hospitalisierung_Inzidenz', ascending=False)

In [ ]:
def plot_hi(data, ymax=None):
    """Plot HospitalisierungsInzidenz in Germany. 
    
    data is a pandas Dataframe with index:
    
    DatetimeIndex(['2021-11-24', '2021-11-24', '2021-11-24', '2021-11-24',
               '2021-11-24', '2021-11-24', '2021-11-24', '2021-11-24',
               '2021-11-24', '2021-11-24',
               ...
               '2020-03-05', '2020-03-05', '2020-03-05', '2020-03-05',
               '2020-03-05', '2020-03-05', '2020-03-05', '2020-03-05',
               '2020-03-05', '2020-03-05'],
        dtype='datetime64[ns]', name='Datum', length=10710, freq=None)
               
    and columns:
    
    Index(['Bundesland', '7T_Hospitalisierung_Inzidenz'], dtype='object')
    
    Example Dataframes can be created in `germany-compute-hospitalisierungsrate.ipynb`.
    """
      
    fig, ax = plt.subplots(figsize=(8, 10))

    # plot whole of Germany separately
    land = "Bundesgebiet"
    land_data = data[data['Bundesland'] == land]
    # use shortcut for the Data (d) we care about
    d = land_data['7T_Hospitalisierung_Inzidenz']
    
    ax.plot(d.index, d, 
            linewidth=5, alpha=0.9, label=land)

    # and all other states ("Länder")
    bundeslaender = set(data['Bundesland']) - {"Bundesgebiet"}
    for land in bundeslaender:
        land_data = data[data['Bundesland'] == land]
        d = land_data['7T_Hospitalisierung_Inzidenz']
    
        ax.plot(d.index, d, '.-', label=land,)
        # label last data point
        ax.text(d.index[0], d[0], land, rotation=45)

        
    ax.legend(loc='center', bbox_to_anchor=(1.3, 0.5))
    fig.autofmt_xdate()
    ax.grid()
    ax.margins(0) # remove default margins (matplotlib verision 2+)

    
    min_date = data.index[-1]
    label_pos_x = min_date + datetime.timedelta(days=1)
    
    props = dict(boxstyle='round', facecolor='white', alpha=1.0)
    
    ax.axhspan(0, 3, facecolor='green', alpha=0.2)
    ax.text(label_pos_x, 4, "2G", bbox=props)
    ax.axhspan(3, 6, facecolor='yellow', alpha=0.2)
    ax.text(label_pos_x, 7, "2G+", bbox=props)
    ax.axhspan(6, 9, facecolor='orange', alpha=0.2)
    ax.text(label_pos_x, 10, "restrictions", bbox=props)
    ax.axhspan(9, 12, facecolor='red', alpha=0.2)
    ax.text(label_pos_x, 13, "additional measures required?", bbox=props)
    ax.axhspan(9, data['7T_Hospitalisierung_Inzidenz'].max(), facecolor='purple', alpha=0.1)

    if ymax:
        ax.set_ylim(top=ymax)
    
    return fig, ax

## Example usage:
# fig, ax = plot_hi(data["2021-10-01":])


Possible extensions:

- show also intensive care bed capacities (because the hospitalisierungsrate is  only about the number of new people going to hospital: it gives no information about the situation in the hospital)

- explore correlation between infections / hospitalisierungsrate / deaths / vaccination status

